In [ ]:
import super_gradients as sg
import torchinfo
import torch

print(f'{torch.cuda.device_count()} GPU found: {torch.cuda.get_device_name("cuda")}')

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL_ARCH = 'yolo_nas_s'

# Basic inference using pre-trained weights
# Model Zoo - https://github.com/Deci-AI/super-gradients/blob/master/documentation/source/model_zoo.md
model = sg.training.models.get(MODEL_ARCH, pretrained_weights='coco').to(DEVICE)

torchinfo.summary(model=model, input_size=(16, 3, 640, 640),
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width=20,
        row_settings=['var_names']
)

In [ ]:
# Inference on an image
model.predict('https://blog.jetbrains.com/wp-content/uploads/2022/01/image1-1.png', conf=0.25).show()

'''
# Inference on a video
INPUT_VIDEO_PATH = 'test.mp4'
OUTPUT_VIDEO_PATH = 'detection.mp4'
model.predict(INPUT_VIDEO_PATH).save(OUTPUT_VIDEO_PATH)
'''

In [ ]:
!kaggle datasets download -p datasets -d nadinpethiyagoda/vehicle-dataset-for-yolo --unzip

BATCH_SIZE = 16
WORKERS = 8 # Roughly equal to the CPU core count
EPOCHS = 10

dataset_params = {
    'data_dir': 'datasets/vehicle dataset',
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    'test_images_dir':'valid/images',
    'test_labels_dir':'valid/labels',
    'classes': ['car', 'threewheel', 'bus', 'truck', 'motorbike', 'van']
}

train_data = sg.training.dataloaders.dataloaders.coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': WORKERS
    }
)

val_data = sg.training.dataloaders.dataloaders.coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': WORKERS
    }
)

test_data = sg.training.dataloaders.dataloaders.coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': WORKERS
    }
)

print('Data augmentation:')
print(train_data.dataset.transforms)
print('Dataloader parameters:')
print(train_data.dataloader_params)
print('Dataset parameters')
print(train_data.dataset.dataset_params)

In [ ]:
train_data.dataset.plot(plot_transformed_data=False)

In [ ]:
train_data.dataset.plot(plot_transformed_data=True)

In [ ]:
# Fine-tuning a pre-trained model
model = sg.training.models.get(MODEL_ARCH, num_classes=len(dataset_params['classes']), pretrained_weights='coco').to(DEVICE)
trainer = sg.training.Trainer(experiment_name='yolo-nas', ckpt_root_dir='checkpoints')

train_params = {}
train_params['silent_mode'] = False
train_params['average_best_models'] = True
train_params['initial_lr'] = 5e-4
train_params['lr_mode'] = 'cosine'
train_params['cosine_final_lr_ratio'] = 0.1
train_params['optimizer'] = 'Adam'
train_params['optimizer_params'] = {'weight_decay': 1e-4}
train_params['max_epochs'] = EPOCHS
train_params['loss'] = sg.training.losses.PPYoloELoss(use_static_assigner=False, num_classes=len(dataset_params['classes']), reg_max=16)
train_params['metric_to_watch'] = 'mAP@0.50'

train_params['valid_metrics_list'] = [
        sg.training.metrics.DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=sg.training.models.detection_models.pp_yolo_e.PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ]

trainer.train(model=model, training_params=train_params, train_loader=train_data, valid_loader=val_data)

In [ ]:
# Testing the best model on the test set
best_model = sg.training.models.get(sg.common.object_names.Models.YOLO_NAS_S, num_classes=len(dataset_params['classes']), checkpoint_path='checkpoints/yolo-nas/average_model.pth')

callback = sg.training.models.detection_models.pp_yolo_e.PPYoloEPostPredictionCallback(score_threshold=0.01, nms_top_k=1000, max_predictions=300, nms_threshold=0.7)
test_metrics = sg.training.metrics.DetectionMetrics_050(score_thres=0.1, top_k_predictions=300, num_cls=len(dataset_params['classes']), normalize_targets=True, post_prediction_callback=callback)
trainer.test(model=best_model, test_loader=test_data, test_metrics_list=test_metrics)